# SGCN National List

The full [national list](https://www1.usgs.gov/csas/swap/national_list.html) of SGCN species across 2005 and 2015 represents a relatively complex query that needs to sum up the total states reporting each species. There may be some way to drive everything with some feature of the Elasticsearch index on the full original data that I haven't figured out yet, but I was only able to come up with a SQL statement to drive this.
```sql
SELECT concat('http://services.itis.gov/?q=tsn:', t.itis->'tsn') AS taxonomicauthorityid,
(array_agg(t.itis->'nameWInd'))[1] AS scientificname,
(array_agg(Coalesce(t.itis->'vernacular:English',s.commonname_submitted)))[1] AS commonname,
(array_agg(s.taxonomicgroup_submitted ORDER BY sgcn_year DESC))[1] AS taxonomicgroup,
sum(((s.sgcn_year = 2005))::integer) AS sgcn2005,
sum(((s.sgcn_year = 2015))::integer) AS sgcn2015
FROM sgcn.sgcn s
JOIN tir.tir2 t ON
t.registration->'SGCN_ScientificName_Submitted' = s.scientificname_submitted
WHERE t.itis->'itisMatchMethod' NOT LIKE 'NotMatched%'
GROUP BY t.itis->'tsn'
```  
Running that live is way too costly on the system, so I built a view in GC2 using this select statement and indexed that in Elasticsearch as sgcn_nationallist. This results in a much more responsive query. This query selects only those records where there is an accepted taxonomic authority ID, which is the basic definition of what ends up on the national list.

## UPDATE
The whole SGCN system has been completely reengineered, but I tried to keep the basic final output in something close to the state that has been built against so far for the SWAP app. The sgcn_nationallist view and ElasticSearch index should be identical to what they were before, but the underlying data are all new. Here are a couple of caveats:

* The query starts from the standpoint of the Taxonomic Information Registry joining to the SGCN table on the submitted/registered name.
* Common Name comes from the ITIS vernacular English name if it exists or else uses the submitted common name from one of the states.
* The taxonomic group still comes from what the states originally submitted, so it is blank for some entries. This will be improved once Abby provides a mapping from ITIS taxonomic levels to some logical grouping that we want to put the national list into.
* The underlying data from the states is also all new here. I built a whole new process that reads directly from the source data repository in ScienceBase and processes source files into records in the new sgcn.sgcn table (new sgcn schema in the GC2 instance). Those are then processed using a different method of checking taxonomy against name authorities. Currently, the final data only include the most solid matches on ITIS. WoRMS taxonomic checks have not been completed to fill in some of the blanks, and the ITIS matching algorithm can be improved to find additional matches. I took a fairly conservative approach on the matching process, so there will likely be additional matches found in future to expand out the "SGCN National List."
* The taxonomic authority ID is concatenated to a URL string that can serve as a usable link for machine access if we want to put more ITIS information together. This ID is always the final accepted TSN for the name matched to ITIS and from which we draw taxonomy, common names, and other properties.
* This query will have to be redone once I get the WoRMS matching service running and we need to account for taxonomic matches on more than one authority.

In [1]:
import requests
from IPython.display import display

In [2]:
#Class to render tables
class ListTable(list):
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

This query returns results from the Elasticsearch index for the sgcn_nationallist view. It only calls the first 25 results, so that will need to be paginated for the SWAP online app. I included the taxonomic authority ID as a reference. Those IDs to ITIS or WoRMS return a machine-readable response and are not content negotiable, so if we want to include them in the UI, we would need to translate the ID into something for humans.

In [8]:
sgcnNationalListURL = "https://gc2.mapcentia.com/api/v1/elasticsearch/search/bcb/sgcn/sgcn_nationallist?size=25&q={%22sort%22:[{%22properties.scientificname%22:{%22order%22:%22asc%22}}]}"
sgcnNationalList = requests.get(sgcnNationalListURL).json()

tableNationalList = ListTable()
tableNationalList.append(['Scientific Name', 'Common Name', '2005', '2015', 'Taxonomic Group', 'Taxonomic Authority ID/Link'])

for hit in sgcnNationalList['hits']['hits']:
    tableNationalList.append([hit['_source']['properties']['scientificname'], hit['_source']['properties']['commonname'], hit['_source']['properties']['sgcn2005'], hit['_source']['properties']['sgcn2015'], hit['_source']['properties']['taxonomicgroup'], hit['_source']['properties']['taxonomicauthorityid']])

display(tableNationalList)

Scientific Name,Common Name,2005,2015,Taxonomic Group,Taxonomic Authority ID/Link
Archeolarca aalbui,Aalbu's Cave Pseudoscorpion,1,0,Arachnids,http://services.itis.gov/q=tsn:749263
Uria aalge,Common Murre,3,1,Birds,http://services.itis.gov/q=tsn:176974
Pyrgulopsis aardahli,Benton Valley springsnail,1,0,Gastropods,http://services.itis.gov/q=tsn:568181
Poanes aaroni aaroni,Aarons Skipper,0,1,Insects,http://services.itis.gov/q=tsn:707326
Abacion tesselatum,A millipede,1,1,Other Invertebrates,http://services.itis.gov/q=tsn:570281
Farancia abacura abacura,Eastern Mud Snake,1,1,Reptiles,http://services.itis.gov/q=tsn:174165
Farancia abacura,Mud Snake,3,1,Reptiles,http://services.itis.gov/q=tsn:174164
Farancia abacura reinwardtii,Western Mud Snake,3,2,Reptiles,http://services.itis.gov/q=tsn:209185
Abaeis nicippe,Sleepy Orange,0,1,Insects,http://services.itis.gov/q=tsn:778216
Abagrotis brunneipennis,Yankee Dart,0,1,Insects,http://services.itis.gov/q=tsn:771341
